<a href="https://colab.research.google.com/github/ajeetth/Multimodal-RAG-for-Drone-Technology/blob/main/RAG_for_Drone_tech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating an Index-Based Deep Lake Vector Store for Semantic Search with LlamaIndex and OpenAI

Building a Semantic Search Engine with Deep Lake, LlamaIndex, and OpenAI:

*   Installing the Environment
*   Creating and populating the Vector Store &   dataset
*   Getting started with  index-based semantic search




In [1]:
!pip install llama-index-vector-stores-deeplake==0.1.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.1.2
    Uninstalling tenacity-9.1.2:
      Successfully uninstalled tenacity-9.1.2
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


In [2]:
!pip install deeplake==3.9.18

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.9/608.9 kB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of s3transfer to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 12.3 MB/s eta 0:00:00
   ━━━

In [1]:
!pip install llama-index==0.10.64

INFO: pip is looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of llama-index-readers-llama-parse to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to 

In [ ]:
# !pip install openai==1.66.3

In [2]:
from google.colab import userdata
api_key = userdata.get('OPENAI_KEY')
deeplake_token = userdata.get('ACTIVELOOP_TOKEN')

In [3]:
import openai
openai.api_key = api_key

In [4]:
import os
os.environ['ACTIVELOOP_TOKEN'] = deeplake_token


# Pipeline 1: Collecting and Preparing Documents

In [5]:
urls = [
    "https://github.com/VisDrone/VisDrone-Dataset",
    "https://paperswithcode.com/dataset/visdrone",
    "https://openaccess.thecvf.com/content_ECCVW_2018/papers/11133/Zhu_VisDrone-DET2018_The_Vision_Meets_Drone_Object_Detection_in_Image_Challenge_ECCVW_2018_paper.pdf",
    "https://github.com/VisDrone/VisDrone2018-MOT-toolkit",
    "https://en.wikipedia.org/wiki/Object_detection",
    "https://en.wikipedia.org/wiki/Computer_vision",
    "https://en.wikipedia.org/wiki/Convolutional_neural_network",
    "https://en.wikipedia.org/wiki/Unmanned_aerial_vehicle",
    "https://www.faa.gov/uas/",
    "https://www.tensorflow.org/",
    "https://pytorch.org/",
    "https://keras.io/",
    "https://arxiv.org/abs/1804.06985",
    "https://arxiv.org/abs/2202.11983",
    "https://motchallenge.net/",
    "http://www.cvlibs.net/datasets/kitti/",
    "https://www.dronedeploy.com/",
    "https://www.dji.com/",
    "https://arxiv.org/",
    "https://openaccess.thecvf.com/",
    "https://roboflow.com/",
    "https://www.kaggle.com/",
    "https://paperswithcode.com/",
    "https://github.com/"
]

In [6]:
!mkdir data

In [7]:
import requests
import re
import os
from bs4 import BeautifulSoup

def clean_text(content):
    # Remove references and unwanted characters
    content = re.sub(r'\[\d+\]', '', content)   # Remove references
    content = re.sub(r'[^\w\s\.]', '', content)  # Remove punctuation (except periods)
    return content

def fetch_and_clean(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad responses (e.g., 404)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Prioritize "mw-parser-output" but fall back to "content" class if not found
        content = soup.find('div', {'class': 'mw-parser-output'}) or soup.find('div', {'id': 'content'})
        if content is None:
            return None

        # Remove specific sections, including nested ones
        for section_title in ['References', 'Bibliography', 'External links', 'See also', 'Notes']:
            section = content.find('span', id=section_title)
            while section:
                for sib in section.parent.find_next_siblings():
                    sib.decompose()
                section.parent.decompose()
                section = content.find('span', id=section_title)

        # Extract and clean text
        text = content.get_text(separator=' ', strip=True)
        text = clean_text(text)
        return text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching content from {url}: {e}")
        return None  # Return None on error

# Directory to store the output files
output_dir = './data/'  # More descriptive name
os.makedirs(output_dir, exist_ok=True)

# Processing each URL (and skipping invalid ones)
for url in urls:
    article_name = url.split('/')[-1].replace('.html', '')  # Handle .html extension
    filename = os.path.join(output_dir, f"{article_name}.txt")

    clean_article_text = fetch_and_clean(url)
    if clean_article_text:  # Only write to file if content exists
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(clean_article_text)

print(f"Content(ones that were possible) written to files in the '{output_dir}' directory.")

Content(ones that were possible) written to files in the './data/' directory.


In [8]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document
from llama_index.vector_stores.deeplake import DeepLakeVectorStore

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.11/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
/usr/local/lib/python3.11/dist-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (4.1.18) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


# Load the documents from ./data/

In [9]:
documents = SimpleDirectoryReader("./data/").load_data()

In [ ]:
documents[0]

Document(id_='d6c1c9b9-3488-4c08-abd4-24de65170bb5', embedding=None, metadata={'file_path': '/content/data/1804.06985.txt', 'file_name': '1804.06985.txt', 'file_type': 'text/plain', 'file_size': 3798, 'creation_date': '2025-03-16', 'last_modified_date': '2025-03-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='High Energy Physics  Theory arXiv1804.06985 hepth Submitted on 19 Apr 2018 Title A Near Horizon Extreme Binary Black Hole Geometry Authors Jacob Ciafre  Maria J. Rodriguez View a PDF of the paper titled A Near Horizon Extreme Binary Black Hole Geometry by Jacob Ciafre and Maria J. Rodriguez View PDF Abstract A new solution of fourdimensional vacuum General Relativity is presented. It describes the near horizon region of the extreme maximally s

# Pipeline 2 : Creating and Populating a Deep Lake Vector store

In [25]:
from llama_index.core import StorageContext

vector_store_path = "hub://ajithdevadiga/drone_dataV2"
dataset_path = "hub://ajithdevadiga/drone_dataV2"

vector_store = DeepLakeVectorStore(dataset_path=dataset_path, overwrite=True)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

Your Deep Lake dataset has been successfully created!


Uploading data to deeplake dataset.


100%|██████████| 87/87 [00:00<00:00, 167.72it/s]
/

Dataset(path='hub://ajithdevadiga/drone_dataV2', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (87, 1)      str     None   
 metadata     json      (87, 1)      str     None   
 embedding  embedding  (87, 1536)  float32   None   
    id        text      (87, 1)      str     None   


In [26]:
import deeplake
ds = deeplake.load('hub://ajithdevadiga/drone_dataV2')

\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/ajithdevadiga/drone_dataV2



|

hub://ajithdevadiga/drone_dataV2 loaded successfully.



In [27]:
import json
import pandas as pd
import numpy as np

# Assuming 'ds' is your loaded Deep Lake dataset

# Create a dictionary to hold the data
data = {}

# Iterate through the tensors in the dataset
for tensor_name in ds.tensors:
    tensor_data = ds[tensor_name].numpy()

    # Check if the tensor is multi-dimensional
    if tensor_data.ndim > 1:
        # Flatten multi-dimensional tensors
        data[tensor_name] = [np.array(e).flatten().tolist() for e in tensor_data]
    else:
        # Convert 1D tensors directly to lists and decode text
        if tensor_name == "text":
            data[tensor_name] = [t.tobytes().decode('utf-8') if t else "" for t in tensor_data]
        else:
            data[tensor_name] = tensor_data.tolist()

# Create a Pandas DataFrame from the dictionary
df = pd.DataFrame(data)

In [28]:
def display_record(record_number):
    record = df.iloc[record_number]
    display_data = {
        "ID": record.get("id", "N/A"),
        "Metadata": record.get("metadata", "N/A"),
        "Text": record.get("text", "N/A"),
        "Embedding": record.get("embedding", "N/A")
    }

    # Print the ID
    print("ID:")
    print(display_data["ID"])
    print()

    # Print the metadata in a structured format
    print("Metadata:")
    metadata = display_data["Metadata"]
    if isinstance(metadata, list):
        for item in metadata:
            for key, value in item.items():
                print(f"{key}: {value}")
            print()
    else:
        print(metadata)
    print()

    # Print the text
    print("Text:")
    print(display_data["Text"])
    print()

    # Print the embedding
    print("Embedding:")
    print(display_data["Embedding"])
    print()

# Function call to display a record
rec = 0  # Replace with the desired record number
display_record(rec)

ID:
['3bbca82d-f631-4ccb-ab6a-d579e52896de']

Metadata:
file_path: /content/data/1804.06985.txt
file_name: 1804.06985.txt
file_type: text/plain
file_size: 3798
creation_date: 2025-04-10
last_modified_date: 2025-04-10
_node_content: {"id_": "3bbca82d-f631-4ccb-ab6a-d579e52896de", "embedding": null, "metadata": {"file_path": "/content/data/1804.06985.txt", "file_name": "1804.06985.txt", "file_type": "text/plain", "file_size": 3798, "creation_date": "2025-04-10", "last_modified_date": "2025-04-10"}, "excluded_embed_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "excluded_llm_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "relationships": {"1": {"node_id": "a4f42f7b-1f0a-4ee0-a488-5f3c0eaddfa3", "node_type": "4", "metadata": {"file_path": "/content/data/1804.06985.txt", "file_name": "1804.06985.txt", "file_type": "text/plain", "file_size": 3798, "cre

In [30]:
rec = 4  # Replace with the desired record number
display_record(rec)

ID:
['10be25cb-e8c2-4447-b7a1-9f6e5156ddc2']

Metadata:
file_path: /content/data/Computer_vision.txt
file_name: Computer_vision.txt
file_type: text/plain
file_size: 54866
creation_date: 2025-04-10
last_modified_date: 2025-04-10
_node_content: {"id_": "10be25cb-e8c2-4447-b7a1-9f6e5156ddc2", "embedding": null, "metadata": {"file_path": "/content/data/Computer_vision.txt", "file_name": "Computer_vision.txt", "file_type": "text/plain", "file_size": 54866, "creation_date": "2025-04-10", "last_modified_date": "2025-04-10"}, "excluded_embed_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "excluded_llm_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "relationships": {"1": {"node_id": "b009e4c5-e17d-4f8b-8150-3cbf981423f4", "node_type": "4", "metadata": {"file_path": "/content/data/Computer_vision.txt", "file_name": "Computer_vision.txt", "file_type": "text

In [29]:
rec = 30 # Replace with the desired record number
display_record(rec)

ID:
['5ada79e8-4258-43dd-89a3-738cbe85f3b3']

Metadata:
file_path: /content/data/Convolutional_neural_network.txt
file_name: Convolutional_neural_network.txt
file_type: text/plain
file_size: 107458
creation_date: 2025-04-10
last_modified_date: 2025-04-10
_node_content: {"id_": "5ada79e8-4258-43dd-89a3-738cbe85f3b3", "embedding": null, "metadata": {"file_path": "/content/data/Convolutional_neural_network.txt", "file_name": "Convolutional_neural_network.txt", "file_type": "text/plain", "file_size": 107458, "creation_date": "2025-04-10", "last_modified_date": "2025-04-10"}, "excluded_embed_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "excluded_llm_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "relationships": {"1": {"node_id": "4b4d146a-65be-4e1f-ae1a-029346d8fc88", "node_type": "4", "metadata": {"file_path": "/content/data/Convolutional_neural_n

# Original Documents

In [31]:
# Ensure 'text' column is of type string
df['text'] = df['text'].astype(str)
# Create documents with IDs
documents = [Document(text=row['text'], doc_id=str(row['id'])) for _, row in df.iterrows()]

# Pipeline 3:Index-based RAG

User input and RAG parameters

In [32]:
user_input="How do drones identify vehicles?"

#similarity_top_k
k=3
#temperature
temp=0.1
#num_output
mt=1024

# cosine similarity metric

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [35]:
def calculate_cosine_similarity_with_embeddings(text1, text2):
    embeddings1 = model.encode(text1)
    embeddings2 = model.encode(text2)
    similarity = cosine_similarity([embeddings1], [embeddings2])
    return similarity[0][0]

# Vector store index query engine

In [37]:
from llama_index.core import VectorStoreIndex
vector_store_index = VectorStoreIndex.from_documents(documents)

In [38]:
print(type(vector_store_index))

<class 'llama_index.core.indices.vector_store.base.VectorStoreIndex'>


In [39]:
vector_query_engine = vector_store_index.as_query_engine(similarity_top_k=k, temperature=temp, num_output=mt)

In [40]:
print(type(vector_query_engine))

<class 'llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine'>


# Query Response

In [41]:
import pandas as pd
import textwrap

def index_query(input_query):
    response = vector_query_engine.query(input_query)

    # Optional: Print a formatted view of the response (remove if you don't need it in the output)
    print(textwrap.fill(str(response), 100))

    node_data = []
    for node_with_score in response.source_nodes:
        node = node_with_score.node
        node_info = {
            'Node ID': node.id_,
            'Score': node_with_score.score,
            'Text': node.text
        }
        node_data.append(node_info)

    df = pd.DataFrame(node_data)

    # Instead of printing, return the DataFrame and the response object
    return df, response

In [42]:
import time
#start the timer
start_time = time.time()
df, response = index_query(user_input)
# Stop the timer
end_time = time.time()
# Calculate and print the execution time
elapsed_time = end_time - start_time
print(f"Query execution time: {elapsed_time:.4f} seconds")

print(df.to_markdown(index=False, numalign="left", stralign="left"))

Drones can identify vehicles through various means such as visual sensors, radar systems, lidar
technology, and even artificial intelligence algorithms that help in object recognition and
classification. These technologies enable drones to detect and track vehicles accurately during
surveillance or monitoring operations.
Query execution time: 1.2517 seconds
| Node ID                              | Score    | Text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [43]:
nodeid=response.source_nodes[0].node_id
nodeid

'2fae2ea7-e30e-4b2d-ba8c-eb2d52d55814'

In [44]:
response.source_nodes[0].get_text()


"['7.3. Sensing for Drones  Introduction to Robotics and Perception . www.roboticsbook.org .  7.5. Trajectory Optimization  Introduction to Robotics and Perception . www.roboticsbook.org .  PierreJean Bristeau François Callou David Vissière Nicolas Petit 2011. The Navigation and Control technology inside the AR.Drone micro UAV PDF . IFAC World Congress . Archived PDF from the original on 27 February 2023 . Retrieved 4 February 2016 .  Cellular enables safer drone deployments . Qualcomm . Archived from the original on 9 May 2018 . Retrieved 9 May 2018 .  Identifying Critical MannedUnmanned Teaming Skills for Unmanned Aircraft System Operators PDF . U.S. Army Research Institute for the Behavioral and Social Sciences . September 2012. Archived PDF from the original on 6 February 2016.  Minimum requirements related to technical performance for IMT2020 radio interfaces . www.itu.int . Archived from the original on 6 August 2020 . Retrieved 8 October 2020 .  Vinogradov Evgenii Kumar A. V. S.

# Optimized chunking

In [45]:
# Assuming you have the 'response' object from query_engine.query()

for node_with_score in response.source_nodes:
    node = node_with_score.node  # Extract the Node object from NodeWithScore
    chunk_size = len(node.text)
    print(f"Node ID: {node.id_}, Chunk Size: {chunk_size} characters")

Node ID: 2fae2ea7-e30e-4b2d-ba8c-eb2d52d55814, Chunk Size: 3444 characters
Node ID: ce5432a8-8526-4c9c-a8c4-b4b7ceba199c, Chunk Size: 4074 characters
Node ID: 60c5a8a1-78de-4ff0-867e-4d38e94344a2, Chunk Size: 4799 characters


# Performance metric

In [46]:
import numpy as np

def info_metrics(response):
  # Calculate the performance (handling None scores)
  scores = [node.score for node in response.source_nodes if node.score is not None]
  if scores:  # Check if there are any valid scores
      weights = np.exp(scores) / np.sum(np.exp(scores))
      perf = np.average(scores, weights=weights) / elapsed_time
  else:
      perf = 0  # Or some other default value if all scores are None

  average_score=np.average(scores, weights=weights)
  print(f"Average score: {average_score:.4f}")
  print(f"Query execution time: {elapsed_time:.4f} seconds")
  print(f"Performance metric: {perf:.4f}")

In [47]:
info_metrics(response)

Average score: 0.8283
Query execution time: 1.2517 seconds
Performance metric: 0.6617


In [48]:
similarity_score = calculate_cosine_similarity_with_embeddings(user_input, str(response))
print(f"Cosine Similarity Score: {similarity_score:.3f}")
print(f"Query execution time: {elapsed_time:.4f} seconds")
performance=similarity_score/elapsed_time
print(f"Performance metric: {performance:.4f}")

Cosine Similarity Score: 0.812
Query execution time: 1.2517 seconds
Performance metric: 0.6487


# Tree index query engine

In [49]:
from llama_index.core import TreeIndex
tree_index = TreeIndex.from_documents(documents)

In [50]:
print(type(tree_index))

<class 'llama_index.core.indices.tree.base.TreeIndex'>


In [51]:
tree_query_engine = tree_index.as_query_engine(similarity_top_k=k, temperature=temp, num_output=mt)

In [52]:
import time
import textwrap
# Start the timer
start_time = time.time()
response = tree_query_engine.query(user_input)
# Stop the timer
end_time = time.time()
# Calculate and print the execution time
elapsed_time = end_time - start_time
print(f"Query execution time: {elapsed_time:.4f} seconds")

print(textwrap.fill(str(response), 100))

Query execution time: 3.4169 seconds
Drones identify vehicles by utilizing computer vision technology related to object detection. This
involves detecting instances of semantic objects of a certain class, such as cars, in digital images
and videos. The process may involve using deep neural networks and models trained on datasets like
COCO to detect vehicles based on their visual features and characteristics.


## Performance metric

In [53]:
similarity_score = calculate_cosine_similarity_with_embeddings(user_input, str(response))
print(f"Cosine Similarity Score: {similarity_score:.3f}")
print(f"Query execution time: {elapsed_time:.4f} seconds")
performance=similarity_score/elapsed_time
print(f"Performance metric: {performance:.4f}")

Cosine Similarity Score: 0.770
Query execution time: 3.4169 seconds
Performance metric: 0.2252


# List index query engine

In [54]:
from llama_index.core import ListIndex
list_index = ListIndex.from_documents(documents)

In [55]:
print(type(list_index))

<class 'llama_index.core.indices.list.base.SummaryIndex'>


In [56]:
list_query_engine = list_index.as_query_engine(similarity_top_k=k, temperature=temp, num_output=mt)

In [57]:
#start the timer
start_time = time.time()
response = list_query_engine.query(user_input)
# Stop the timer
end_time = time.time()
# Calculate and print the execution time
elapsed_time = end_time - start_time
print(f"Query execution time: {elapsed_time:.4f} seconds")

print(textwrap.fill(str(response), 100))

Query execution time: 16.8532 seconds
Drones can identify vehicles using computer vision techniques, specifically convolutional neural
networks (CNNs). These networks are trained to process images captured by the drone's cameras and
make predictions based on the features extracted from the images. By analyzing the visual data,
drones can recognize vehicles by detecting specific patterns, shapes, and characteristics associated
with different types of vehicles. This allows drones to perform tasks such as surveillance,
tracking, and object detection in various applications.


## Performance metric


In [58]:
similarity_score = calculate_cosine_similarity_with_embeddings(user_input, str(response))
print(f"Cosine Similarity Score: {similarity_score:.3f}")
print(f"Query execution time: {elapsed_time:.4f} seconds")
performance=similarity_score/elapsed_time
print(f"Performance metric: {performance:.4f}")

Cosine Similarity Score: 0.746
Query execution time: 16.8532 seconds
Performance metric: 0.0442


# Keyword index query index

In [59]:
from llama_index.core import KeywordTableIndex
keyword_index = KeywordTableIndex.from_documents(documents)

In [60]:
# Extract data for DataFrame
data = []
for keyword, doc_ids in keyword_index.index_struct.table.items():
    for doc_id in doc_ids:
        data.append({"Keyword": keyword, "Document ID": doc_id})

# Create the DataFrame
df = pd.DataFrame(data)
df

,Keyword,Document ID
0,black,45c65fcf-bdac-4cdb-8fe6-a914bdca999d
1,binary,45c65fcf-bdac-4cdb-8fe6-a914bdca999d
2,nonsupersymmetric,45c65fcf-bdac-4cdb-8fe6-a914bdca999d
3,hole,45c65fcf-bdac-4cdb-8fe6-a914bdca999d
4,high,8621264d-2784-406e-b126-c2131eb46dce
...,...,...
4322,intelligent drones,329f3523-8fa4-4442-9190-b2159c5cd8ed
4323,magazine,329f3523-8fa4-4442-9190-b2159c5cd8ed
4324,future,329f3523-8fa4-4442-9190-b2159c5cd8ed
4325,quantitative,329f3523-8fa4-4442-9190-b2159c5cd8ed


In [61]:
keyword_query_engine = keyword_index.as_query_engine(similarity_top_k=k, temperature=temp, num_output=mt)

In [ ]:
import time

# Start the timer
start_time = time.time()

# Execute the query (using .query() method)
response = keyword_query_engine.query(user_input)

# Stop the timer
end_time = time.time()

# Calculate and print the execution time
elapsed_time = end_time - start_time
print(f"Query execution time: {elapsed_time:.4f} seconds")

print(textwrap.fill(str(response), 100))

## Performance metric

In [63]:
similarity_score = calculate_cosine_similarity_with_embeddings(user_input, str(response))
print(f"Cosine Similarity Score: {similarity_score:.3f}")
print(f"Query execution time: {elapsed_time:.4f} seconds")
performance=similarity_score/elapsed_time
print(f"Performance metric: {performance:.4f}")

Cosine Similarity Score: 0.746
Query execution time: 16.8532 seconds
Performance metric: 0.0442
